In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from multiprocess import Pool

from learning_utilities import *

Setting environment for Gordon


No vtk
Cannot import mxnet.


In [2]:
stack = 'MD589'
s2f = metadata_cache['sections_to_filenames'][stack]
first_sec, last_sec = metadata_cache['section_limits'][stack]
im_w, im_h = metadata_cache['image_shape'][stack]

In [3]:
x_spacing = 300
y_spacing = 300
superpixel_centers =  np.array([(x, y) 
                                for y in range(200, im_h-200, y_spacing) 
                                for x in range(200, im_w-200, x_spacing)])

superpixel_xmins = superpixel_centers[:,0] - x_spacing/2
superpixel_xmaxs = superpixel_centers[:,0] + x_spacing/2
superpixel_ymins = superpixel_centers[:,1] - y_spacing/2
superpixel_ymaxs = superpixel_centers[:,1] + y_spacing/2

In [4]:
NEWREGION_DIR = '/home/yuncong/csd395/CSHL_newRegions/'

In [ ]:
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1):
    
    bar.value = sec
    
    img_fn = s2f[sec]
    if img_fn in ['Nonexisting', 'Rescan', 'Placeholder']:
        continue
        
    mask_tb = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec)
    
    patch_indices = [[(stack, sec, i) for i in locate_patches_v2(stack=stack, mask_tb=mask_tb, bbox_lossless=[xmin, ymin, xmax+1-xmin, ymax+1-ymin])]
             for xmin, xmax, ymin, ymax in zip(superpixel_xmins, superpixel_xmaxs, superpixel_ymins, superpixel_ymaxs)]

    features = apply_function_to_nested_list(addresses_to_features, patch_indices)
    features = map(np.array, features)
    
    output_dir = os.path.join(NEWREGION_DIR, stack, img_fn)
    
    execute_command('mkdir -p ' + output_dir)
    
    save_hdf_v2(features, os.path.join(output_dir, '%(img_fn)s_features.hdf' % dict(img_fn=img_fn)))

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [29]:
# labelmap = slic(img[5000:6000, 5000:6000], n_segments=100, compactness=1., 
#                 max_iter=1000, sigma=2, convert2lab=False, 
#                 spacing=None, min_size_factor=0.5, max_size_factor=30)
# labelmap_viz = mark_boundaries(img[5000:6000, 5000:6000], labelmap)
# display_image(labelmap_viz)